In [1]:
from bs4 import BeautifulSoup
import requests
import re
import csv

In [2]:
def main():
    datalist = getData()
    write_to_csv(datalist)

In [3]:
# global variable to find variables using regular expression
findTitle = re.compile(r'<a href="/title/(.*)</a>')
findDuration = re.compile(r'<span class="runtime">(.*) min</span>')
findGenre = re.compile(r'<span class="genre">\s*(.*?)\s*</span>')
findRating = re.compile(r'<div class="inline-block ratings-imdb-rating" data-value="(.*)" name="ir">')
findVoteCount = re.compile(r'<span data-value="(.*)" name="nv">(.*)</span>')

In [4]:
# scrape webpage
def getData():
    datalist = [] # store all movies
    # store 400 movies with 50 movies in a page
    for year in range(2000,2021):
        year = str(year)
        for i in range(8):
            baseurl = 'https://www.imdb.com/search/title/?title_type=feature&release_date='+year+'-01-01,'+year+'-12-31&languages=en&sort=num_votes,desc&start='
            url = baseurl + str(i*50+1)
            response = requests.get(url)
            html = response.text
            soup = BeautifulSoup(html, "html.parser")
            items = soup.find_all('div', class_ = 'lister-item-content')
            for item in items:
                data = [] # save all info for one movie
                item = str(item)
                id_title = re.findall(findTitle, item)[0]
                id_title = id_title.split('>',1)
                imdb_id = id_title[0].split('/',1)[0]
                data.append(imdb_id)
                title = id_title[1]
                data.append(title)
                duration = re.findall(findDuration, item)
                # duration may be empty
                if len(duration) != 0:
                    data.append(int(duration[0]))
                else:
                    data.append("")

                genre = re.findall(findGenre, item)
                data.append(genre)
                rating = float(re.findall(findRating, item)[0])
                data.append(rating)
                vote_gross = re.findall(findVoteCount, item)
                vote_count = int(vote_gross[0][0])
                data.append(vote_count)
                # gross may be empty
                if len(vote_gross) > 1:
                    gross = vote_gross[1][0]
                    gross = int(gross.replace(',',''))
                    data.append(gross)
                else:
                    data.append("")

                data.append(year) # insert year column
                datalist.append(data)
            
    return datalist

In [5]:
# write data to csv file
def write_to_csv(datalist):
    header = ['imdb_id','title','duration','genre','rating','vote_count','gross','year']
    
    # opening the csv file in 'w' mode 
    file = open('imdb_USmovie_data.csv', 'w', encoding='utf-8') 

    # writing the data into the file 
    with file:     
        write = csv.writer(file)
        write.writerow(header)
        write.writerows(datalist)

In [ ]:
main()